# Build a Crew to Tailor Job Applications

Build a multi-agent system.  Adapted from CrewAI Lesson 7 of their Initial Course

You will need the following libraries:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

You might need the following libraries.  Some of the libraries might already be installed but the versions are incorrect.  The following section of pip installs are optional.  Hopefully, they'll work for you but if they don't please feel free to leave a comment and I will try to help.

In [ ]:
!pip install openai

In [ ]:
!pip install utils

In [ ]:
!pip install anaconda-cloud-auth --upgrade

In [ ]:
!pip install --force-reinstall -v "Pydantic==2.6.2"

In [ ]:
!pip install Cython

In [ ]:
!pip install --force-reinstall -v "blosc2==2.0.0"

I found it easier to install the following libraries one at a time to allow for debugging any versioning issues.

In [ ]:
!pip install crewai==0.28.8

In [ ]:
!pip install crewai_tools==0.1.6

In [ ]:
!pip install langchain_community==0.0.29

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [ ]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [ ]:
import os
import openai

# save your keys to user/system environment varaibles on your local machine
def get_serper_api_key():
    """Retrieve the Serper API key from an environment variable."""
    api_key = os.getenv('SERPER_API_KEY')
    if not api_key:
        raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")
    return api_key


def get_openai_api_key():
    """Retrieve the OpenAI API key from an environment variable."""
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")
    return api_key

# Retrieve and set the API key
#openai.api_key = get_openai_api_key() 
openai_api_key = get_openai_api_key()

# recommend using gpt-4-turbo model as the results are more human-like, softer, and in the tone of the job posting.
# Hallucinations/accuracy was a mixed bag with 4.0.  In an attempt to be more accurate, the agents completely removed
# 4 years of my experience as a senior project manager :O.  
# I will attempt to fix this with prompting or maybe...
#TO DO: add a QA agent to the team
# Be aware that you PAY FOR this improvement.  $.36 for one run compared to $.80 for 20 runs.

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

#os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [ ]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

resume_name = './jon_verbose_resume.md'
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path=resume_name)
semantic_search_resume = MDXSearchTool(mdx=resume_name)

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

## A brief note about your resume 
When you tailor a resume, I find it is helpful to put everything you can think of in your resume.  Videos and articles will tell you to keep your resumes to the point and that's true but let the AI agent do that for you.  The more facts you can share with the agent the better as it looks for information to tailor towards the job.

In [ ]:
from IPython.display import display, Markdown
# from IPython.display import Markdown, display
display(Markdown(resume_name))

## Creating Agents

In [ ]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [ ]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [ ]:
# Agent 4: Reviewer
reviewer = Agent(
    role="Resume Reviewer",
    goal="Review a given resume and notify the Resume Strategist of any time gaps between positions on the resume. ",
    backstory="You are an expert at reviewing resumes "
              "for the Resume Strategist. "
              "Your goal is to review the resume "
              "to ensure that it follows resume writing best practices.",
    tools = [read_resume, semantic_search_resume,search_tool],
    verbose=True
)

In [ ]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",

    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant experiences and skills, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [ ]:
# Agent 5: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [ ]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key experiences, skills, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "experiences, skills, and qualifications."
    ),
    agent=researcher,
    async_execution=True
)

In [ ]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [ ]:
edit_task = Task(
    description=("Utilize tools to identify the chronoligcal job history for the given resume and "
                 "make sure there are no gaps. You will check for grammatical errors as well."),
    expected_output="The correct chronological order of the job history and a list of of grammatical errors that should be corrected.",   
    agent=reviewer
)

In [ ]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile, job requirements, job history, and grammatical errors obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume ever but "
        "don't make up any information. You must review and update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflect how the candidates "
        "experiences and abilities match the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [ ]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [ ]:
job_application_crew = Crew(
    agents=[researcher
            ,profiler
            ,resume_strategist
            ,reviewer],

    tasks=[research_task
           ,profile_task
           ,resume_strategy_task
           ,edit_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [ ]:
job_application_inputs = {
    'job_posting_url': 'https://docs.google.com/document/d/e/2PACX-1vTeHauCxXtK_E5pcFcjkiizqGKxW6mVrz1jlqU1LM9Jd40COe_PMt9SKd8fPmSFI-lc83w6YatmGeVu/pub',
    'github_url': 'https://github.com/ecocarlisle/',
    'personal_writeup': """Jon is an accomplished Software
    Engineering Leader with 24 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds a MPA and a strong
    background in AI and data science. Jon has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

- Dislplay the generated `tailored_resume.md` file.

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

- Dislplay the generated `interview_materials.md` file.

- now let's get the interviewing materials!

In [ ]:
display(Markdown("./interview_materials.md"))

## if you want to output the tailored resume as PDF you can follow these steps

In [ ]:
### these are some dependencies you will need uncomment these two PIP calls
#!pip install nbconvert
#!pip install pyppeteer

In [ ]:
### in order to use chromium you will need to set this environment variable
os.environ["PYPPETEER_CHROMIUM_REVISION"] = '1263111'

## save your markdown to a new notebook
- open the tailored markdown file and copy/paste into new notebook.  Name the notebook Tailored Resume.
- then run the next line of code

In [ ]:
!jupyter nbconvert --to webpdf --allow-chromium-download Greenway-Health-Director-Dev-Engineering.ipynb

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)